In [14]:
#Import all the libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from functools import reduce
import math
import urllib
import difflib 
import seaborn as sns
import sklearn.model_selection
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
import os
import random
import tensorflow as tf
from keras import backend as K

In [15]:
'''Loads the dataframe of with the features'''
df_All_Companies_merged = pd.read_pickle(r'Data\df_All_Companies_merged_with_features.pkl')

In [16]:
'''
Set the random state for reproducibility of results

'''
def set_random_state():    
    # Seed value
    # Apparently you may use different seed values at each stage
    seed_value= 0

    # 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed_value)

    # 2. Set the `python` built-in pseudo-random generator at a fixed value
    random.seed(seed_value)

    # 3. Set the `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed_value)

    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    tf.compat.v1.set_random_seed(seed_value)

    # 5. Configure a new global `tensorflow` session
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

In [17]:
'''
THIS WILL TRAIN A NEURAL NETWORK USING THE GROUPINGS WE CONSTRUCTED
1. UNIVERISITY RANKING SIGNAL
2. EXPERIENCE SIGNAL
3. CATEGORY OF STARTUP
WE SHALL USE TESTING DATA, VALIDATION DATA, AND SEE THE SUCCESS RATE.
'''


training_params_binary = ['number_of_founders_grouping_scaling',
                          'founders_connectedness_factor_based_on_university_grouping_scaling',
                          'best_ranking_universities_of_founders_grouping_scaling',
                          'best_score_universities_of_founders_grouping_scaling',
                          'average_degree_of_founders_grouping_scaling',
                          'number_of_prev_title_of_founders_grouping_scaling',
                          'managerial_flag_number_of_prev_title_of_founders_grouping_scaling',
                          'top_10_categories_flag',
                          'top_10_category_groups_flag',
                          'top_10_previous_companies_flag',
                          'success_flag']

df_for_binary_model_training = df_All_Companies_merged[training_params_binary].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
print('Size of data set is: ', len(df_for_binary_model_training.index))

y = df_for_binary_model_training.pop('success_flag').to_frame()
print('Size of successful companies in set is: ', (np.sum(y.to_numpy())))
X = df_for_binary_model_training

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.3, random_state = 0)
X_test, X_val, y_test, y_val = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.5, random_state = 0)

X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()
X_val, y_val = X_val.to_numpy(), y_val.to_numpy()

print('Size train set is {} and number of successful companies in set is {}'.format(len(X_train),np.sum(y_train)) )
print('Size test set is {} and number of successful companies in set is {}'.format(len(X_test),np.sum(y_test)) )
print('Size validation set is {} and number of successful companies in set is {}'.format(len(X_val),np.sum(y_val)) )


number_of_classes = 1
number_of_features = X_train.shape[1]
model = Sequential()
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
set_random_state()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 500, verbose = 0)

eval = model.evaluate(x=X_test, y=y_test)
print('The loss on the test set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 1].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only successful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 0].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only unsuccessful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

Size of data set is:  5685
Size of successful companies in set is:  365
Size train set is 3979 and number of successful companies in set is 255
Size test set is 2842 and number of successful companies in set is 182
Size validation set is 2843 and number of successful companies in set is 183
89/89 [==============================] - 0s 1ms/step - loss: 0.1211 - binary_accuracy: 0.9500
The loss on the test set is 0.12109331786632538 and the binary accuracy is 0.9500352144241333
12/12 [==============================] - 0s 1ms/step - loss: 3.6342 - binary_accuracy: 0.1753
The loss on only successful companies set is 3.6342151165008545 and the binary accuracy is 0.17534247040748596
167/167 [==============================] - 0s 1ms/step - loss: 0.0926 - binary_accuracy: 0.9925
The loss on only successful companies set is 0.0925748348236084 and the binary accuracy is 0.9924812316894531


In [18]:
'''
THIS WILL TRAIN A NEURAL NETWORK USING THE GROUPINGS WE CONSTRUCTED
1. UNIVERISITY RANKING SIGNAL
2. EXPERIENCE SIGNAL
3. CATEGORY OF STARTUP
4. NUMBER OF INVERSTORS (NOT INVESTOR SUCCESS)

WE SHALL USE TESTING DATA, VALIDATION DATA, AND SEE THE SUCCESS RATE.
'''


training_params_binary = ['number_of_founders_grouping_scaling',
                          'founders_connectedness_factor_based_on_university_grouping_scaling',
                          'best_ranking_universities_of_founders_grouping_scaling',
                          'best_score_universities_of_founders_grouping_scaling',
                          'average_degree_of_founders_grouping_scaling',
                          'number_of_prev_title_of_founders_grouping_scaling',
                          'managerial_flag_number_of_prev_title_of_founders_grouping_scaling',
                          'top_10_categories_flag',
                          'number_of_investors_grouping_scaling',
                          'top_10_category_groups_flag',
                          'top_10_previous_companies_flag',
                          'success_flag']

df_for_binary_model_training = df_All_Companies_merged[training_params_binary].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
print('Size of data set is: ', len(df_for_binary_model_training.index))

y = df_for_binary_model_training.pop('success_flag').to_frame()
print('Size of successful companies in set is: ', (np.sum(y.to_numpy())))
X = df_for_binary_model_training

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.3, random_state = 0)
X_test, X_val, y_test, y_val = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.5, random_state = 0)

X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()
X_val, y_val = X_val.to_numpy(), y_val.to_numpy()

print('Size train set is {} and number of successful companies in set is {}'.format(len(X_train),np.sum(y_train)) )
print('Size test set is {} and number of successful companies in set is {}'.format(len(X_test),np.sum(y_test)) )
print('Size validation set is {} and number of successful companies in set is {}'.format(len(X_val),np.sum(y_val)) )


number_of_classes = 1
number_of_features = X_train.shape[1]
model = Sequential()
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
set_random_state()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 500, verbose = 0)

eval = model.evaluate(x=X_test, y=y_test)
print('The loss on the test set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 1].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only successful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 0].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only successful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

Size of data set is:  2940
Size of successful companies in set is:  336
Size train set is 2058 and number of successful companies in set is 235
Size test set is 1470 and number of successful companies in set is 168
Size validation set is 1470 and number of successful companies in set is 168
46/46 [==============================] - 0s 908us/step - loss: 0.0703 - binary_accuracy: 0.9585
The loss on the test set is 0.07030022889375687 and the binary accuracy is 0.9585034251213074
11/11 [==============================] - 0s 997us/step - loss: 4.2191 - binary_accuracy: 0.5208
The loss on only successful companies set is 4.219130992889404 and the binary accuracy is 0.5208333134651184
82/82 [==============================] - 0s 827us/step - loss: 0.1228 - binary_accuracy: 0.9816
The loss on only successful companies set is 0.12279508262872696 and the binary accuracy is 0.981566846370697


In [19]:
'''
THIS WILL TRAIN A NEURAL NETWORK USING THE GROUPINGS WE CONSTRUCTED
1. UNIVERISITY RANKING SIGNAL
2. EXPERIENCE SIGNAL
3. CATEGORY OF STARTUP
4. NUMBER OF INVERSTORS (AND INVESTOR SUCCESS)

WE SHALL USE TESTING DATA, VALIDATION DATA, AND SEE THE SUCCESS RATE.
'''


training_params_binary = ['number_of_founders_grouping_scaling',
                          'founders_connectedness_factor_based_on_university_grouping_scaling',
                          'best_ranking_universities_of_founders_grouping_scaling',
                          'best_score_universities_of_founders_grouping_scaling',
                          'average_degree_of_founders_grouping_scaling',
                          'number_of_prev_title_of_founders_grouping_scaling',
                          'managerial_flag_number_of_prev_title_of_founders_grouping_scaling',
                          'top_10_categories_flag',
                          'number_of_investors_grouping_scaling',
                          'average_investor_success',
                          'top_10_category_groups_flag',
                          'top_10_previous_companies_flag',
                          'success_flag']

df_for_binary_model_training = df_All_Companies_merged[training_params_binary].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
print('Size of data set is: ', len(df_for_binary_model_training.index))

y = df_for_binary_model_training.pop('success_flag').to_frame()
print('Size of successful companies in set is: ', (np.sum(y.to_numpy())))
X = df_for_binary_model_training

X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.3, random_state = 0)
X_test, X_val, y_test, y_val = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.5, random_state = 0)

X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()
X_val, y_val = X_val.to_numpy(), y_val.to_numpy()

print('Size train set is {} and number of successful companies in set is {}'.format(len(X_train),np.sum(y_train)) )
print('Size test set is {} and number of successful companies in set is {}'.format(len(X_test),np.sum(y_test)) )
print('Size validation set is {} and number of successful companies in set is {}'.format(len(X_val),np.sum(y_val)) )


number_of_classes = 1
number_of_features = X_train.shape[1]
model = Sequential()
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
set_random_state()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 500, verbose = 0)

eval = model.evaluate(x=X_test, y=y_test)
print('The loss on the test set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 1].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only successful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 0].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only unsuccessful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

Size of data set is:  2940
Size of successful companies in set is:  336
Size train set is 2058 and number of successful companies in set is 235
Size test set is 1470 and number of successful companies in set is 168
Size validation set is 1470 and number of successful companies in set is 168
46/46 [==============================] - 0s 866us/step - loss: 0.0017 - binary_accuracy: 1.0000
The loss on the test set is 0.0017450465820729733 and the binary accuracy is 1.0
11/11 [==============================] - 0s 1ms/step - loss: 0.5651 - binary_accuracy: 0.9286
The loss on only successful companies set is 0.5650855898857117 and the binary accuracy is 0.9285714030265808
82/82 [==============================] - 0s 988us/step - loss: 0.0447 - binary_accuracy: 0.9950
The loss on only successful companies set is 0.044740449637174606 and the binary accuracy is 0.9950076937675476


In [20]:
'''
THIS WILL TRAIN A NEURAL NETWORK USING THE GROUPINGS WE CONSTRUCTED
1. UNIVERISITY RANKING SIGNAL
2. EXPERIENCE SIGNAL
3. CATEGORY OF STARTUP
WE SHALL USE TESTING DATA, VALIDATION DATA, AND SEE THE SUCCESS RATE.
'''


training_params_binary = ['number_of_founders_grouping_scaling',
                          'best_ranking_universities_of_founders_grouping_scaling',
                          'best_score_universities_of_founders_grouping_scaling',
                          'average_degree_of_founders_grouping_scaling',
                          'number_of_prev_title_of_founders_grouping_scaling',
                          'managerial_flag_number_of_prev_title_of_founders_grouping_scaling',
                          'top_10_categories_flag',
                          'top_10_category_groups_flag',
                          'top_10_previous_companies_flag',
                          'success_flag']

df_for_binary_model_training = df_All_Companies_merged[training_params_binary].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
print('Size of data set is: ', len(df_for_binary_model_training.index))

y = df_for_binary_model_training.pop('success_flag').to_frame()
print('Size of successful companies in set is: ', (np.sum(y.to_numpy())))
X = df_for_binary_model_training

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.3, random_state = 0)
X_test, X_val, y_test, y_val = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.5, random_state = 0)

X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()
X_val, y_val = X_val.to_numpy(), y_val.to_numpy()

print('Size train set is {} and number of successful companies in set is {}'.format(len(X_train),np.sum(y_train)) )
print('Size test set is {} and number of successful companies in set is {}'.format(len(X_test),np.sum(y_test)) )
print('Size validation set is {} and number of successful companies in set is {}'.format(len(X_val),np.sum(y_val)) )


number_of_classes = 1
number_of_features = X_train.shape[1]
model = Sequential()
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
set_random_state()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 500, verbose = 0)

eval = model.evaluate(x=X_test, y=y_test)
print('The loss on the test set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 1].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only successful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 0].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only unsuccessful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

Size of data set is:  10305
Size of successful companies in set is:  482
Size train set is 7213 and number of successful companies in set is 337
Size test set is 5152 and number of successful companies in set is 241
Size validation set is 5153 and number of successful companies in set is 241
161/161 [==============================] - 0s 1ms/step - loss: 0.1315 - binary_accuracy: 0.9561
The loss on the test set is 0.13147087395191193 and the binary accuracy is 0.9561335444450378
16/16 [==============================] - 0s 1ms/step - loss: 4.9537 - binary_accuracy: 0.0602
The loss on only successful companies set is 4.953701019287109 and the binary accuracy is 0.060165975242853165
307/307 [==============================] - 0s 915us/step - loss: 0.0503 - binary_accuracy: 0.9984
The loss on only unsuccessful companies set is 0.050279807299375534 and the binary accuracy is 0.9983711838722229


In [21]:
'''
THIS WILL TRAIN A NEURAL NETWORK USING THE GROUPINGS WE CONSTRUCTED
1. UNIVERISITY RANKING SIGNAL
2. EXPERIENCE SIGNAL
3. CATEGORY OF STARTUP
4. NUMBER OF INVERSTORS (NOT INVESTOR SUCCESS)

WE SHALL USE TESTING DATA, VALIDATION DATA, AND SEE THE SUCCESS RATE.
'''


training_params_binary = ['number_of_founders_grouping_scaling',
                          'best_ranking_universities_of_founders_grouping_scaling',
                          'best_score_universities_of_founders_grouping_scaling',
                          'average_degree_of_founders_grouping_scaling',
                          'number_of_prev_title_of_founders_grouping_scaling',
                          'managerial_flag_number_of_prev_title_of_founders_grouping_scaling',
                          'top_10_categories_flag',
                          'number_of_investors_grouping_scaling',
                          'top_10_category_groups_flag',
                          'top_10_previous_companies_flag',
                          'success_flag']

df_for_binary_model_training = df_All_Companies_merged[training_params_binary].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
print('Size of data set is: ', len(df_for_binary_model_training.index))

y = df_for_binary_model_training.pop('success_flag').to_frame()
print('Size of successful companies in set is: ', (np.sum(y.to_numpy())))
X = df_for_binary_model_training

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.3, random_state = 0)
X_test, X_val, y_test, y_val = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.5, random_state = 0)

X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()
X_val, y_val = X_val.to_numpy(), y_val.to_numpy()

print('Size train set is {} and number of successful companies in set is {}'.format(len(X_train),np.sum(y_train)) )
print('Size test set is {} and number of successful companies in set is {}'.format(len(X_test),np.sum(y_test)) )
print('Size validation set is {} and number of successful companies in set is {}'.format(len(X_val),np.sum(y_val)) )


number_of_classes = 1
number_of_features = X_train.shape[1]
model = Sequential()
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
set_random_state()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 500, verbose = 0)

eval = model.evaluate(x=X_test, y=y_test)
print('The loss on the test set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 1].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only successful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 0].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only successful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

Size of data set is:  4687
Size of successful companies in set is:  437
Size train set is 3280 and number of successful companies in set is 306
Size test set is 2343 and number of successful companies in set is 218
Size validation set is 2344 and number of successful companies in set is 219
74/74 [==============================] - 0s 930us/step - loss: 0.1166 - binary_accuracy: 0.9449
The loss on the test set is 0.11661003530025482 and the binary accuracy is 0.9449423551559448
14/14 [==============================] - 0s 1ms/step - loss: 5.0006 - binary_accuracy: 0.3478
The loss on only successful companies set is 5.000645160675049 and the binary accuracy is 0.3478260934352875
133/133 [==============================] - 0s 858us/step - loss: 0.1203 - binary_accuracy: 0.9868
The loss on only successful companies set is 0.12026509642601013 and the binary accuracy is 0.986823558807373


In [22]:
'''
THIS WILL TRAIN A NEURAL NETWORK USING THE GROUPINGS WE CONSTRUCTED
1. UNIVERISITY RANKING SIGNAL
2. EXPERIENCE SIGNAL
3. CATEGORY OF STARTUP
4. NUMBER OF INVERSTORS (AND INVESTOR SUCCESS)

WE SHALL USE TESTING DATA, VALIDATION DATA, AND SEE THE SUCCESS RATE.
'''


training_params_binary = ['number_of_founders_grouping_scaling',
                          'best_ranking_universities_of_founders_grouping_scaling',
                          'best_score_universities_of_founders_grouping_scaling',
                          'average_degree_of_founders_grouping_scaling',
                          'number_of_prev_title_of_founders_grouping_scaling',
                          'managerial_flag_number_of_prev_title_of_founders_grouping_scaling',
                          'top_10_categories_flag',
                          'number_of_investors_grouping_scaling',
                          'average_investor_success',
                          'top_10_category_groups_flag',
                          'top_10_previous_companies_flag',
                          'success_flag']

df_for_binary_model_training = df_All_Companies_merged[training_params_binary].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
print('Size of data set is: ', len(df_for_binary_model_training.index))

y = df_for_binary_model_training.pop('success_flag').to_frame()
print('Size of successful companies in set is: ', (np.sum(y.to_numpy())))
X = df_for_binary_model_training

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.3, random_state = 0)
X_test, X_val, y_test, y_val = sklearn.model_selection.train_test_split(X, y,stratify=y, test_size=0.5, random_state = 0)

X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()
X_val, y_val = X_val.to_numpy(), y_val.to_numpy()

print('Size train set is {} and number of successful companies in set is {}'.format(len(X_train),np.sum(y_train)) )
print('Size test set is {} and number of successful companies in set is {}'.format(len(X_test),np.sum(y_test)) )
print('Size validation set is {} and number of successful companies in set is {}'.format(len(X_val),np.sum(y_val)) )


number_of_classes = 1
number_of_features = X_train.shape[1]
model = Sequential()
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(2**7, input_dim=number_of_features, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
set_random_state()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 500, verbose = 0)

eval = model.evaluate(x=X_test, y=y_test)
print('The loss on the test set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 1].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only successful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

df_for_binary_model_training = df_All_Companies_merged[training_params_binary][df_All_Companies_merged['success_flag'] == 0].copy()
df_for_binary_model_training = df_for_binary_model_training.dropna()
y = df_for_binary_model_training.pop('success_flag').to_frame()
X = df_for_binary_model_training

eval = model.evaluate(x=X.to_numpy(), y=y.to_numpy())
print('The loss on only unsuccessful companies set is {} and the binary accuracy is {}'.format(eval[0],eval[1]))

Size of data set is:  4687
Size of successful companies in set is:  437
Size train set is 3280 and number of successful companies in set is 306
Size test set is 2343 and number of successful companies in set is 218
Size validation set is 2344 and number of successful companies in set is 219
74/74 [==============================] - 0s 1ms/step - loss: 0.0011 - binary_accuracy: 0.9987
The loss on the test set is 0.0010925526730716228 and the binary accuracy is 0.9987195730209351
14/14 [==============================] - 0s 1ms/step - loss: 0.4518 - binary_accuracy: 0.9657
The loss on only successful companies set is 0.45184430480003357 and the binary accuracy is 0.9656750559806824
133/133 [==============================] - 0s 1ms/step - loss: 0.0824 - binary_accuracy: 0.9920
The loss on only unsuccessful companies set is 0.08243946731090546 and the binary accuracy is 0.9919999837875366
